[View in Colaboratory](https://colab.research.google.com/github/RayleighKim/M09_Intro_To_Regression/blob/master/Hackathon1_Stock_Prediction.ipynb)

# Hackathon 1

## Stock Prediction

### Your name :     

### Kaggle Link for exercise :
* [Stock Price trend Prediction dataset](https://www.kaggle.com/aumashe/stock-ew/home) : Historical stock prices of Edwards Lifesciences Corporation (EW) 2000-03-26 ~ 2017-09-09

### Real Data for exercise :
* [내가 원하는 회사 데이터 다운 받을 곳!](https://finance.yahoo.com/quote/005930.KS/history?p=005930.KS&.tsrc=fin-srch-v1)
![how-to](https://github.com/RayleighKim/Example_datasets/blob/master/yahoo_finance.PNG?raw=true)

#### 문제를 정의할 것
* 무엇을 할지
* 왜 하는 것인지
* 어떻게 활용할 것인지

#### 목표
* 성능! 성능! 성능!
* (Optional I) 그냥 투자 했다면 얼마를 벌었을까
* (Optional II) 내 모델을 이용해 투자를 했다면 얼마를 벌었을까

#### 조건
* (권장, not필수) 실습시간에 사용한 데이터 말고 다른 회사 데이터 사용할 것
* Feature Engineering을 통해 여러 변수들을 만들어 사용할 것 : Pandas의 Rolling 기능 적극 활용
* 어떤 모델을 가져다가 써도 좋다. Scikit Learn을 사용하기를 권장함.
* (Optional) Keras를 이용해 시계열 모델링에 도전해봐도 좋다. **GPU를 사용하자**

---------------
Rayleigh Kim @ D:plus


In [0]:
'''
matplolib inline 명령어를 통해서
matplot으로 그리는 플롯들을 주피터 노트북 내에서 볼 수 있게 해준다.
포맷을 retina로 바꾸면 그래프의 화질이 훨씬 좋아진다.
'''
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras import backend as k

from sklearn.metrics import mean_squared_error, r2_score


## Load and Prepare the data

In [0]:
### 여러분의 데이터를 구해서 사용하기를 권장합니다.
### 여러분의 데이터를 업로드 / 구글드라이브 마운트하여 사용하세요.

data_path = 'https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/Stock_Edwards_Lifesciences_corporation.csv'
stock = pd.read_csv(data_path)

In [0]:
# '''
# Stock_Edwards_Lifesciences_corportation.csv 를 업로드 하자.
# '''
# from google.colab import files
# files.upload

In [0]:
# stock = pd.read_csv('Stock_Edwards_Lifesciences_corporation.csv')

In [0]:
print(stock.shape)
stock.head()

In [0]:
stock[:240].plot(x='Date', y='Close')

## Your Feature Engineering!

Feature Engineering은 위대하다. 시계열 모델링 방법을 잘 몰라도, 데이터에 반영해낼 수 있다!

이곳에 Feature Engineering 계획을 세울 것
* ex> 하루 전의 종가를 가져와 사용할 것이다.
* ex> 삼일 전의 시가를 가져와 사용할 것이다.
* ex> 10일 전의 최고가를 가져와 사용할 것이다.
* ex> 과거 20일의 이동평균을 가져와 사용할 것이다.

**Pandas Rolling 기능 사용시 주의**

* min_periods 적절히 활용할 것
* shift를 적절히 활용할 것.
* NA, NaN 등이 처음 row들에 등장할 수 있다.
    * df.fillna() 를 활용하거나
    * 아예 처음 row들을 버려버릴 수 있음.

In [0]:
## Your final Data

print(stock.columns)
stock.dtypes

## Drop columns

* (필수)사용하지 않을 변수들은 제거하자.
* (권장)컬럼을 사용하기 좋은 순서대로 바꾸자.
* (필수) 우리는 Close를 예측할 것이므로 가능하면 close 컬럼을 맨 뒤로 미루자.

다음의 코드를 이용하면 타이핑의 수고를 줄일 수 있다.

```
stock.columns
```



In [0]:
### 코드를 수정하여 사용할 것.

fields_to_drop = ['Date', 'Adj_Close']
data = stock.drop(fields_to_drop, axis=1)
data = data[['Open', 'High', 'Low', 'Volume','Close']]

data.head()

## Scaling Variables

반드시 하기를 권장

사실, 날짜 빼고는 다 해야 함


In [0]:
quant_features = list(data)
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

In [0]:
'''
위에서 쓰인 Dictionary 잠깐 짚어보기.
Dictionary의 각각의 원소는 key : value 쌍으로 되어 있다.
'''

scaled_features

### Pandas Dataframe to Numpy array



In [0]:
xy = data.values
print(xy[:5])

## Modeling!

특별한 문제가 없다면 다음의 셀은 자동으로 실행이 되어야 한다.

* 선택지1(권장) : Feature Engineering만으로 승부를 본다. 위에서 Feature Engineering 해가며 아래 셀을 실행하여 성능을 관찰한다.
* 선택지2 : 실습파일을 참고하여 LSTM 모델링까지 해본다.
* 선택지3(권장) : Scikit-Learn을 이용하여 여러 모델들을 사용해본다. 

In [0]:
k.clear_session()


x_data = xy[:-1]
y_data = xy[1:, [-1]]

x_train, x_valid, x_test = x_data[:-30], x_data[-30:-10], x_data[-10:]
y_train, y_valid, y_test = y_data[:-30], y_data[-30:-10], y_data[-10:]

### modeling
model = Sequential()
model.add(Dense(units =   1, input_dim=x_data.shape[1]))
model.compile(loss='mse', optimizer = 'adam', metrics = ['mse'] )

### Training


model.fit(x_train, y_train, batch_size = 512,
         epochs = 1000, validation_data=(x_valid, y_valid),
          callbacks=[EarlyStopping(monitor='val_loss', patience=15)]
         )

### evaluate

pred_train = model.predict(x_train)
pred_valid = model.predict(x_valid)
pred_test = model.predict(x_test)

print("Mean Squared Error on Training set : {0:.5f}".format(mean_squared_error(y_train, pred_train)  ))
print("Mean Squared Error on Validation set : {0:.5f}".format(mean_squared_error(y_valid, pred_valid)  ))
print("Mean Squared Error on Test set : {0:.5f}".format(mean_squared_error(y_test, pred_test)  ))

print("R-squared Score on Training set : {0:.5f}".format(r2_score(y_train, pred_train)))
print("R-squared Score on Validation set : {0:.5f}".format(r2_score(y_valid, pred_valid)))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(y_test, pred_test)))

fig, ax = plt.subplots(figsize=(10,5))

# 예측값도 Scaling 되어 있으므로 그것을 원래대로 되돌려주는 과정
mean, std = scaled_features['Close']

ax.plot((pred_test)*std+mean, label = 'Prediction')
ax.plot((y_test)*std+mean, label = 'Real')
ax.set_xlim(right = len(y_test))
ax.legend()
